Задание 1
Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:
x - последовательность цифр
y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10

Задача:
научить модель предсказывать y(i) по x(i)
пробовать RNN, LSTM, GRU

LSTM

In [ ]:
import torch
from torch import nn
import re
import random
import tqdm
import time
import numpy as np

In [ ]:
import random
x = [random.randint(0, 10) for _ in range(1000)]
x[:10]

[2, 1, 3, 8, 8, 1, 4, 4, 3, 8]

In [ ]:
def find_y(x):
  y = [0] * len(x)
  for i in range(len(x)):
     y[i] = x[i]+x[1]
     if y[i]>=10:
      y[i]-=10
  return y

In [ ]:
y = find_y(x)
y[:10]

[3, 2, 4, 9, 9, 2, 5, 5, 4, 9]

In [ ]:
len(x), len(y), x[0], y[0]

(1000, 1000, 2, 3)

In [ ]:
# Преобразуем в тензоры
X = torch.tensor(x, dtype=torch.float32).view(-1, 1, 1)  # (samples, time_step, features)
Y = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # (samples, output)

In [ ]:
X[0], Y[0]

(tensor([[2.]]), tensor([3.]))

In [ ]:
# 2. Определение модели LSTM
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.fc = nn.Linear(50, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Берем последний выход LSTM
        return out

In [ ]:
# Создаем модель
model = LSTMModel()

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, Y)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 6.2388
Epoch [200/1000], Loss: 3.8152
Epoch [300/1000], Loss: 1.8804
Epoch [400/1000], Loss: 1.3230
Epoch [500/1000], Loss: 1.0659
Epoch [600/1000], Loss: 0.8654
Epoch [700/1000], Loss: 0.6928
Epoch [800/1000], Loss: 0.5400
Epoch [900/1000], Loss: 0.3230
Epoch [1000/1000], Loss: 0.0797


In [ ]:
# 4. Предсказание значений
model.eval()
y_pred = []
with torch.no_grad():
    for i in range(len(y)):
      test_input = X[i].view(1, 1, 1)  # Последние 10 значений для предсказания
      predicted = model(test_input).item()
      y_pred.append(int(predicted))

In [ ]:
def check_accuracy(y, y_pred):
  count = 0
  for i in range(len(y)):
    if y[i]==y_pred[i]:
      count+=1
  return count/len(y)

In [ ]:
check_accuracy(y, y_pred)

0.545

RNN

In [ ]:
class RNNModel(nn.Module):
    def __init__(self):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.fc = nn.Linear(50, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Берем последний выход RNN
        return out

# Создаем модель
model = RNNModel()

In [ ]:
# 3. Обучение модели
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, Y)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 6.3207
Epoch [200/1000], Loss: 3.4610
Epoch [300/1000], Loss: 1.6775
Epoch [400/1000], Loss: 1.2057
Epoch [500/1000], Loss: 0.8425
Epoch [600/1000], Loss: 0.5340
Epoch [700/1000], Loss: 0.3067
Epoch [800/1000], Loss: 0.1781
Epoch [900/1000], Loss: 0.0992
Epoch [1000/1000], Loss: 0.0544


In [ ]:
# 4. Предсказание значений
model.eval()
y_pred = []
with torch.no_grad():
    for i in range(len(y)):
      test_input = X[i].view(1, 1, 1)
      predicted = model(test_input).item()
      y_pred.append(int(predicted))

In [ ]:
check_accuracy(y, y_pred)

0.544

GRU

In [ ]:
# 2. Определение модели GRU
class GRUModel(nn.Module):
    def __init__(self):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.fc = nn.Linear(50, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])  # Берем последний выход GRU
        return out

# Создаем модель
model = GRUModel()

In [ ]:
# 3. Обучение модели
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, Y)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 5.9173
Epoch [200/1000], Loss: 2.6628
Epoch [300/1000], Loss: 0.9671
Epoch [400/1000], Loss: 0.4402
Epoch [500/1000], Loss: 0.1694
Epoch [600/1000], Loss: 0.0938
Epoch [700/1000], Loss: 0.0565
Epoch [800/1000], Loss: 0.0030
Epoch [900/1000], Loss: 0.0005
Epoch [1000/1000], Loss: 0.0002


In [ ]:
# 4. Предсказание значений
model.eval()
y_pred = []
with torch.no_grad():
    for i in range(len(y)):
      test_input = X[i].view(1, 1, 1)
      predicted = model(test_input).item()
      y_pred.append(int(predicted))

In [ ]:
check_accuracy(y, y_pred)

0.553